# Final Project - Advanced Topics in Computer Science 8
Yehonatan Peisakhovsky yehonatan-pe@campus.technion.ac.il ?????????
Ido Zuckerman ido.z@campus.technion.ac.il 323102830

## Topic
?????

## Introduction
?????

## Research Question
?????

## Assumptions
?????


## Methods and Experiments
????? (expiremnts and metrics)

## Results

In [ ]:
from enviorment import Env
from utils import *
from sklearn.model_selection import KFold
from sklearn.metrics import ndcg_score
from tqdm import tqdm


#### Hyperparameters

In [ ]:
amount_of_queries = 50
k = 5
Epochs = 3

#### Preprocess

In [ ]:
docs, queries, rel = create_documents_queries_and_relevence()
queries_with_relevant_docs = list(rel.keys())
queries = {k: queries[k] for k in queries_with_relevant_docs}
docs, queries = preprocess_data(docs, queries)

#### Example

In [ ]:
# Chosen queries to work with and do cross validation
chosen_queries = [x for x in queries.keys()][:amount_of_queries]
cv = KFold(k)
# original ndcg without changing the documents
undisturbed_ndcg = []
# Ndcg after the change
disturbed_ndcg = []
for train_indices, test_indices in tqdm(cv.split(chosen_queries)):
        # The ids of the queries used to train,
        # they actually matter just when using rank svm
        train_queries_ids = [chosen_queries[x] for x in train_indices]
        # The ids of the queries used to test the model
        test_queries_ids = [chosen_queries[x] for x in test_indices]
        train_queries = {k: queries[k] for k in train_queries_ids}
        train_rel = {k: rel[k] for k in train_queries_ids}
        test_queries = {k: queries[k] for k in test_queries_ids}
        test_rel = {k: rel[k] for k in test_queries_ids}
        # Assigns random values to the queries, we can change seed
        queries_worth = create_queries_worth(test_queries)
        # Environment initialization
        environment = Env(docs = docs,strategic_percentage=0.1,model_type="tf_idf",number_of_word_to_add=1000,train_queries=train_queries,train_relevance_ranking=train_rel,seed = 10)
        # Epochs represents the amount of time we change which agents are chosen to
        # be strategic. We need to talk about how they are chosen, because now they are random
        # meaning if we run too many epochs then the ndcg with or without corruption will
        # be the same (because sometimes strategic agents are relevant and pushed up, sometimes
        # not relevant are pushed up.
        for i in range(Epochs):
            environment.change_config()
            # Scores of unchanged documents and real ndcg
            queries_scores = environment.run(test_queries)
            for query_id, scores in queries_scores.items():
                gt = np.array([[1 if i in test_rel[query_id] else 0 for i in docs.keys()]])
                undisturbed_ndcg.append(ndcg_score(gt, scores))
            # Makes strategic agents change their documents
            environment.corrupt(test_queries, queries_worth)
            # Scores after documents change
            queries_scores = environment.run(test_queries)
            for query_id, scores in queries_scores.items():
                gt = np.array([[1 if i in test_rel[query_id] else 0 for i in docs.keys()]])
                disturbed_ndcg.append(ndcg_score(gt, scores))

### changing the percentage of strategic agents

In [ ]:
docs = ...

for model in ["okapi_bm25", "tf_idf", "rank_svm"]:
    for strategic_per in np.arange(0, 1.1, 0.1):
        env = env(docs, strategic_per, model, number_of_word_to_add=5)

### changing the number of words to add

In [ ]:
for model in ["okapi_bm25", "tf_idf", "rank_svm"]:
    for number_of_word_to_add in range(1, 11):
        env = env(100, 0.5, model, number_of_word_to_add=number_of_word_to_add)

## Discussion
?????

## Limitations
?????